# Try Fastai

Modifying mask using:
https://forums.fast.ai/t/how-to-load-multiple-classes-of-rle-strings-from-csv-severstal-steel-competition/51445

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

from pathlib import Path
import os
import random

from fastai.callbacks.hooks import *
from fastai.utils.mem import *

import random

## Preparation

Main challenge with fastai is the creation of databunch. After that its all straightforward.

In [ ]:
PROJ_DIR = Path(os.path.abspath("")).parent
PROJ_DIR

In [ ]:
path_train_img = PROJ_DIR / "data" / "raw" / "train_images"
path_train_labels = PROJ_DIR / "data" / "raw" / "train_labels"
path_test_img = PROJ_DIR / "data" / "raw" / "test_images"

In [ ]:
fnames = get_image_files(path_train_img)
fnames[:3]

In [ ]:
img_f = random.choice(fnames)
img = open_image(img_f)
img.show(figsize=(5, 5))

In [ ]:
df_labels = pd.read_csv(PROJ_DIR / "data/raw/train.csv")
df_labels.head(10)

### Reading Labels

In [ ]:
shape = np.array(img.shape[-2:])
shape

## Create DataBunch

In [ ]:
free = gpu_mem_get_free_no_cache()
# the max size of bs depends on the available GPU RAM
if free > 8200:
    bs = 8
else:
    bs = 4
print(f"using bs={bs}, have {free}MB of GPU RAM free")

In [ ]:
src_size = np.array(img.shape[1:])
src_size

In [ ]:
size = src_size // 4
bs = 2 
size

In [ ]:
src = (SegmentationItemList.from_folder(path_train_img)
       .split_by_rand_pct()
       .label_from_func(get_y_fn, classes=[0, 252, 253, 254, 255]))
src

In [ ]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))
data

In [ ]:
data.show_batch(2, figsize=(5,5))

In [ ]:
data.show_batch(2, figsize=(5, 5), ds_type=DatasetType.Valid)

In [ ]:
wd=1e-2

In [ ]:
learn = unet_learner(data, models.resnet34, wd=wd)

In [ ]:
lr_find(learn)